# Logging traces 🔗

Single logs can be useful to tell what's going on at a given time and place in the code, or to define custom metrics.

However, they don't provide a full picture of the behavior of the solution, i.e. the sequence of events that led to a given output.

This is where traces come in. In this notebook, we'll extend our solution to log and inspect traces in the cloud.

## Understanding traces

Trace logs are a sequence of logs that are related to each other. Each individual log is a JSON object with a `trace_id` field that identifies the trace it belongs to. Let's explore a bit how this looks like.

In [1]:
import time

from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor, ConsoleSpanExporter

In [ ]:
# trace.set_tracer_provider(TracerProvider())
# tracer_provider = trace.get_tracer_provider()
# cloud_trace_exporter = ConsoleSpanExporter()
# batch_span_processor = BatchSpanProcessor(cloud_trace_exporter)
# tracer_provider.add_span_processor(batch_span_processor)

tracer = trace.get_tracer(__name__)

# Note: The traces will appear right below this cell

Here we wrap some functions with our `tracer`, so they produce trace logs once called.

In [3]:
@tracer.start_as_current_span("do_something_else")
def do_something_else():
    time.sleep(2)

@tracer.start_as_current_span("do_something")
def do_something():
    time.sleep(1)
    do_something_else()

do_something()  # Check out the traces that will appear above!

{
    "name": "do_something_else",
    "context": {
        "trace_id": "0xe878725629aa6e151176ac57a1d3a4bf",
        "span_id": "0xccfb7f920ba457c9",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x83b9cb315e3f9478",
    "start_time": "2025-12-17T12:05:19.799898Z",
    "end_time": "2025-12-17T12:05:21.804136Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {},
    "events": [],
    "links": [],
    "resource": {
        "attributes": {
            "telemetry.sdk.language": "python",
            "telemetry.sdk.name": "opentelemetry",
            "telemetry.sdk.version": "1.25.0",
            "service.name": "unknown_service"
        },
        "schema_url": ""
    }
}
{
    "name": "do_something",
    "context": {
        "trace_id": "0xe878725629aa6e151176ac57a1d3a4bf",
        "span_id": "0x83b9cb315e3f9478",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2

## Extend solution and explore traces in cloud

Let's extend our solution so it produces trace logs, so we can inspect these in the cloud.

> **Exercise** 📝
>
> - Fill in the TODO's `# TODO(12-logging-traces)` in the `extraction` module so our solution produces trace logs that get sent to the cloud.
> - Re-run the app and trigger some traces to be logged.
> - Now, navigate to Application Insights in Azure Monitor and explore the traces you just logged under the "Transaction search" or "End-to-end transaction details" views.
>
> 💡 Hint:
>
> - The `tracer` is already configured to send logs to Azure. See how `tracer` is configured in `logs.py`.
> - You can filter for traces in Application Insights, e.g. using operation name or custom dimensions.
> - To filter only your service's traces, you can add a `{"service.name": <your-user-name>}` dictionary to the trace's attributes. 
>
>For example (assuming you imported the .env user name as USER_NAME earlier):
> ```python
> @tracer.start_as_current_span(
>        "extract_general_info", 
>        attributes={"service.name": USER_NAME}
>        )
>```



## Adding logs as part of traces

Traces become even more powerful when we add logs to them. This way we can fully inspect the sequence of events, including inputs and outputs at each step, that led to a given output (or later feedback from the user).

> **Exercise** 📝
>
> - Explore the functions `log_with_trace` and `log_extraction_step`, which are imported in the `extraction` module.
> - Use these functions to fill in the TODO's `# TODO(12-log-with-trace)` in the `extraction` module, so we send useful logs along with our traces.
> - Re-run the app and trigger some traces to be logged.
> - Now, navigate to where you can see the traces in Azure Monitor and explore the traces you just logged. (ask trainer to show you if you can't find this)
> - Can you see your logs in the traces? 🎉
>
> 💡 Hint:
>
> - For the `event` argument in `log_extraction_step`, you can pass the span name, given by `trace.get_current_span().name`
> - For the `output` argument, use `output.model_dump()` to get a string representation of the model output.
> - A log statement for logging extraction info as part of the trace would something like:
>   ```python
>   log_extraction_step(trace.get_current_span().name, article, prompt_template, output.model_dump(), business)
>   ```